# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [3]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [4]:
import random
from itertools import permutations
import numpy as np

In [5]:
def count_edges(edges, cut):
    ans = 0
    for edge in edges:
        if not(set(edge) <= cut[0] or set(edge) <= cut[1]):
            ans += 1
    return ans

In [5]:
def get_random_cut(vert):
    part = set(random.sample(vert, len(vert) // 2))
    cut = [part, vert - part]
    return cut

In [6]:
def find_nearest(edges, cut, k):
    test = [set(), set()]
    ans = [set(), set()]
    best_dist = count_edges(edges, cut)
    for first in permutations(cut[0], k):
        for second in permutations(cut[1], k):
            pool_set = set(first).union(set(second))
            for i in permutations(pool_set, k):
                left = set(i)
                right = pool_set.difference(left)
                test[0] = cut[0].union(left).difference(right)
                test[1] = cut[1].union(right).difference(left)
                dist = count_edges(edges, test)
                if dist < best_dist:
                    best_dist, ans = dist, test.copy()
    return ans

In [6]:
def basic_local_search(graph):
    vertices = graph[0]
    edges = graph[1]
    new_edges = [[] for i in range(len(graph[0]) + 1)]
    for edge in edges:
        new_edges[edge[0]].append(edge[1])
        new_edges[edge[1]].append(edge[0])
    cut = [set(range(1, len(graph[0]) // 2 + 1)), vertices.difference(set(range(1, len(graph[0]) // 2 + 1)))]
    
    np_cut = np.zeros(len(vertices) + 1)
    np_cut[list(cut[1])] = 1
    
    sum_vert = np.zeros(len(vertices) + 1)
    for v in range(1, len(vertices) + 1):
        sum_vert[v] = np_cut[new_edges[v]].sum()
    cnt_edge = np.zeros(len(vertices) + 1)
    for v in vertices:
        cnt_edge[v] = len(new_edges[v])
    current = count_edges(edges, cut)
    while True:
        res = False
        cur_dist = current
        for i in range(1):
            ans = [0, 0]
            best_dist = 99999999999999
            for first in cut[0]:
                for second in cut[1]:
                    sum_first = sum_vert[first]
                    sum_second = sum_vert[second]
                    dist = cur_dist + (cnt_edge[first] - sum_first) - sum_first + sum_second - (cnt_edge[second] - sum_second)
                    if dist < best_dist:
                        best_dist, ans = dist, [first, second]
            cut[0].add(ans[1])
            cut[0].remove(ans[0])
            cut[1].add(ans[0])
            cut[1].remove(ans[1])
            np_cut[ans[1]] = 0
            np_cut[ans[0]] = 1
            sum_vert[new_edges[ans[1]]] -= 1
            sum_vert[new_edges[ans[0]]] += 1
            if best_dist < current:
                current = best_dist
                res = True
                break
            else:
                cur_dist = best_dist
        if res == False:
            break
    return cut[0]

In [7]:
print(basic_local_search(read_col_file("myciel3.col")))

{1, 4, 5, 6, 8}


In [8]:
print(basic_local_search(read_col_file("myciel4.col")))

{2, 4, 7, 8, 9, 10, 11, 12, 16, 17, 22}


In [9]:
print(basic_local_search(read_col_file("myciel5.col")))

{2, 6, 7, 8, 10, 11, 13, 14, 15, 18, 19, 20, 21, 22, 24, 25, 26, 28, 29, 33, 34, 45, 46}


In [ ]:
print(basic_local_search(read_col_file("myciel6.col")))

In [ ]:
print(basic_local_search(read_col_file("myciel7.col")))